# Data Preparation
L'obiettivo di questa fase è preparare i dati per la modellazione, includendo:
- Pulizia dei dati
- Feature engineering
- Encoding delle variabili categoriche
- Scaling delle variabili numeriche
- Feature selection

---


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Caricamento del dataset
data = pd.read_csv('heart_disease.csv')

# 0. Conversione di Yes/No in 1/0
Le colonne con valori `Yes` e `No` vengono trasformate in valori numerici `1` e `0` per l'analisi.

In [3]:
# Identificazione delle colonne con valori Yes/No
yes_no_columns = ['Heart Disease Status', 'Smoking', 'Family Heart Disease', 'Diabetes', 
                  'High Blood Pressure', 'Low HDL Cholesterol', 'High LDL Cholesterol']

# Conversione Yes/No -> 1/0
for col in yes_no_columns:
    data[col] = data[col].map({'Yes': 1, 'No': 0})


# 1. Gestione dei Valori Mancanti
In questa sezione, gestiamo i valori mancanti:
- Per variabili numeriche: Imputazione con la media.
- Per variabili categoriche:
  - Per `Alcohol Consumption`, creiamo una nuova categoria "Unknown".
  - Per le altre, imputiamo con la moda.

In [4]:
# Imputazione dei valori mancanti per variabili numeriche
numerical_columns = ['Age', 'Blood Pressure', 'Cholesterol Level', 'BMI', 
                     'Triglyceride Level', 'Fasting Blood Sugar', 'CRP Level', 'Homocysteine Level', 'Sleep Hours']

for col in numerical_columns:
    # Usare la media preserva i valori numerici e riduce l'impatto dei valori mancanti
    data[col].fillna(data[col].mean(), inplace=True)

# Creazione della categoria "Unknown" per Alcohol Consumption
# Motivazione: Il 26% dei dati mancanti è significativo; trattarli come una categoria separata
# permette di non distorcere la distribuzione esistente.
data['Alcohol Consumption'] = data['Alcohol Consumption'].fillna('Unknown')

# Imputazione con la moda per altre variabili categoriche
categorical_columns = ['Gender', 'Exercise Habits', 'Smoking', 'Family Heart Disease', 
                       'Diabetes', 'High Blood Pressure', 'Low HDL Cholesterol', 
                       'High LDL Cholesterol', 'Stress Level', 'Sugar Consumption']

for col in categorical_columns:
    # Usare la moda è una scelta semplice per preservare la distribuzione dei dati categorici
    data[col].fillna(data[col].mode()[0], inplace=True)


C:\Users\giu30\AppData\Local\Temp\ipykernel_13656\551113754.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplace=True)
C:\Users\giu30\AppData\Local\Temp\ipykernel_13656\551113754.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

# 2. Gestione degli Outlier
Gli outlier vengono trattati utilizzando il metodo di capping, limitando i valori estremi ai limiti IQR.
Questo metodo riduce l'impatto degli outlier senza rimuovere record utili.


In [5]:
# Trattamento degli outlier
for col in numerical_columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[col] = data[col].clip(lower=lower_bound, upper=upper_bound)


# 3. Feature Engineering
Creazione di nuove feature:
- **Cholesterol_BMI_Ratio**: Rapporto tra colesterolo e BMI.
- **Stress_Sleep_Index**: Indice che combina il livello di stress e le ore di sonno.


In [6]:
# Creazione di nuove feature
data['Cholesterol_BMI_Ratio'] = data['Cholesterol Level'] / data['BMI']
data['Stress_Sleep_Index'] = data['Stress Level'].map({'Low': 1, 'Medium': 2, 'High': 3}) / data['Sleep Hours']

# 4. Encoding delle Variabili Categoriali
- **One-Hot Encoding** per variabili categoriche senza ordine.
- **Label Encoding** per variabili ordinali.


In [7]:
from sklearn.preprocessing import LabelEncoder

# One-Hot Encoding per variabili categoriche senza ordine
# Scelta: Converte le categorie in colonne binarie, mantenendo la variabilità
data = pd.get_dummies(data, columns=['Gender', 'Exercise Habits', 'Smoking', 
                                     'Family Heart Disease', 'Alcohol Consumption'], drop_first=True)

# Label Encoding per variabili categoriche ordinali
# Scelta: Mantiene l'ordine delle categorie (es. Low < Medium < High)
ordinal_columns = ['Stress Level', 'Sugar Consumption']

le = LabelEncoder()
for col in ordinal_columns:
    data[col] = le.fit_transform(data[col])

# Verifica finale
print("Dati dopo l'encoding:")
print(data.head())


Dati dopo l'encoding:
    Age  Blood Pressure  Cholesterol Level  Diabetes        BMI  \
0  56.0           153.0              155.0       0.0  24.991591   
1  69.0           146.0              286.0       1.0  25.221799   
2  46.0           126.0              216.0       0.0  29.855447   
3  32.0           122.0              293.0       0.0  24.130477   
4  60.0           166.0              242.0       1.0  20.486289   

   High Blood Pressure  Low HDL Cholesterol  High LDL Cholesterol  \
0                  1.0                  1.0                   0.0   
1                  0.0                  1.0                   0.0   
2                  0.0                  1.0                   1.0   
3                  1.0                  0.0                   1.0   
4                  1.0                  0.0                   0.0   

   Stress Level  Sleep Hours  ...  Cholesterol_BMI_Ratio  Stress_Sleep_Index  \
0             2     7.633228  ...               6.202086            0.262012   


# 5. Scaling delle Variabili Numeriche
Le variabili numeriche vengono standardizzate per normalizzarne i range.

In [8]:
# Standardizzazione delle variabili numeriche
scaler = StandardScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# 6. Feature Selection
Viene utilizzato un modello Random Forest per identificare le feature più rilevanti.

In [9]:
# Feature importance con Random Forest
X = data.drop('Heart Disease Status', axis=1)
y = data['Heart Disease Status']

rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)

importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("Feature Importances:\n", importances)


Feature Importances:
 Homocysteine Level             0.082072
CRP Level                      0.081803
BMI                            0.080264
Sleep Hours                    0.079622
Cholesterol_BMI_Ratio          0.078634
Stress_Sleep_Index             0.077934
Triglyceride Level             0.076785
Cholesterol Level              0.071034
Fasting Blood Sugar            0.069916
Age                            0.068912
Blood Pressure                 0.067032
Sugar Consumption              0.019216
Stress Level                   0.014000
Diabetes                       0.012423
Low HDL Cholesterol            0.012177
High Blood Pressure            0.011960
Smoking_1.0                    0.011956
Family Heart Disease_1.0       0.011703
High LDL Cholesterol           0.011579
Exercise Habits_Low            0.010871
Gender_Male                    0.010627
Exercise Habits_Medium         0.010605
Alcohol Consumption_Low        0.009802
Alcohol Consumption_Unknown    0.009688
Alcohol Consumptio

# 8. Salvataggio del Dataset
Il dataset preparato viene salvato per la fase di modellazione.


In [10]:
# Salvataggio del dataset
data.to_csv('prepared_heart_disease_dataset.csv', index=False)
print("Dataset preparato salvato con successo.")

Dataset preparato salvato con successo.
